<div style="background-color: yellow; padding: 10px; border-radius: 25px;">
    <h1 style="color: black; text-align: center;">Sepsis Early Warning System</h1>
</div>

# 1. Introduction to the Dataset

* **Source:**  
    - PhysioNet, restricted dataset accessible after completing a course online. The dataset I used for this project is derived from the MIMIC-IV database. 
    - It is a comprehensive medical dataset containing health records of hospitalized patients.
    - The dataset contains patient data, including demographics, vital signs, lab test results, and more.
    
```Access to this dataset required completing an online course on patient privacy and data use, ensuring adherence to strict ethical guidelines.```
    

# 2. Summary of the Final Dataset Structure

### **Final Dataset Structure**

| **Column**                           | **Source Table**                          | **Description**                                |
|--------------------------------------|-------------------------------------------|------------------------------------------------|
| `subject_id`                         | All Tables                                | Unique patient identifier                     |
| `hadm_id`               | All Tables            | Hospital admission ID                         |
| `gender`                | `patients.csv`        | Patient gender                                |
| `anchor_age`            | `patients.csv`        | Patient age                                   |
| `admission_type`        | `admissions.csv`      | Type of admission                             |
| `admission_location`    | `admissions.csv`      | Location of admission                         |
| `los`                   | `icustays.csv`        | ICU length of stay                            |
| `hospital_stay_hours`   | `admissions.csv`      | Total hospital stay duration                  |
| `icu_stay_hours`        | `icustays.csv`        | ICU stay duration in hours                    |
| `Bicarbonate`           | `labevents`           | Bicarbonate levels                            |
| `C-Reactive Protein`    | `labevents`           | C-Reactive Protein levels                     |
| `Creatinine`            | `labevents`           | Creatinine levels                             |
| `Lactate`               | `labevents`           | Lactate levels                                |
| `Platelet Count`        | `labevents`           | Platelet count in the blood                   |
| `White Blood Cells`     | `labevents`           | White blood cell count                        |
| `Blood Pressure`        | `chartevents`         | Average blood pressure                        |
| `Brain`                 | `chartevents`         | Brain oxygenation level                       |
| `GCS`                   | `chartevents`         | Glasgow Coma Scale                            |
| `Glucose`               | `chartevents`         | Blood glucose levels                          |
| `Heart Rate`            | `chartevents`         | Heart rate in beats per minute                |
| `Oxygen`                | `chartevents`         | Oxygen support                                |
| `Respiratory`           | `chartevents`         | Respiratory rate                              |
| `Temperature`           | `chartevents`         | Body temperature                              |
| `Sepsis`                | `diagnosis_icd`       | Binary label indicating sepsis (0 or 1)       |

# 3. Features and Target Extraction from the Mimic-IV Database

In [2]:
import pandas as pd

**Tables Used from MIMIC-IV Database:**

1. ```labevents.csv and d_labitems.csv:```   ```17.1gb 158M Rows ```
   * Provided laboratory data like Bicarbonate, Creatinine, Lactate, etc.
    * Metadata helped map item IDs to human-readable labels.

2. ```chartevents.csv and d_chartitems.csv:```   ```39gb 432M Rows ```
    * Included vital signs like Heart Rate, Blood Pressure, and Temperature.
    * Metadata helped identify relevant chart items for sepsis prediction.
3. ```admissions.csv and patients.csv:```
    * Contained demographic and admission-related data (e.g., gender, age, admission_type).
4. ```diagnoses_icd.csv and d_icd_diagnoses.csv:```
    * Helped identify sepsis-related diagnoses using ICD codes.
5. ```icustays.csv:```
    * Provided ICU-specific details like first_careunit, last_careunit, and icu_stay_hours.

In [81]:
print("admissions", admissions.columns)
print("patients", patients.columns)
print("d_icd_diagnosis", d_icd_diagnosis.columns)
print("diagnosis_ics", diagnosis_icd.columns)
print("icu_stays", icu_stays.columns)

admissions Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admit_provider_id', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag'],
      dtype='object')
patients Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')
d_icd_diagnosis Index(['icd_code', 'icd_version', 'long_title'], dtype='object')
diagnosis_ics Index(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version'], dtype='object')
icu_stays Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los'],
      dtype='object')


### 1. labevents.csv and d_labitems.csv

In [78]:
labitems = pd.read_csv("d_labitems.csv")

In [67]:
labitems.head(2)

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas


In [68]:
labitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   itemid    1650 non-null   int64 
 1   label     1646 non-null   object
 2   fluid     1650 non-null   object
 3   category  1650 non-null   object
dtypes: int64(1), object(3)
memory usage: 51.7+ KB


In [69]:
requried_lab_features = ['White Blood Cells', 'Lactate', 'Platelet Count', 'Creatinine', 'Bicarbonate', 'C-Reactive Protein']
lab_features = labitems[labitems['label'].isin(requried_lab_features)]

In [70]:
lab_features

,itemid,label,fluid,category
11,50813,Lactate,Blood,Blood Gas
80,50882,Bicarbonate,Blood,Chemistry
87,50889,C-Reactive Protein,Blood,Chemistry
110,50912,Creatinine,Blood,Chemistry
450,51265,Platelet Count,Blood,Hematology
486,51301,White Blood Cells,Blood,Hematology
872,51755,White Blood Cells,Blood,Chemistry
873,51756,White Blood Cells,Blood,Chemistry
1516,52442,Lactate,Blood,Blood Gas
1521,52546,Creatinine,Blood,Chemistry


In [71]:
lab_features_itemid = [50813, 50882, 50889, 50912, 51265, 51301, 51755, 51756, 52442, 52546, 53154, 53189]

**Extracting Lab Features**

In [57]:
use = ['labevent_id']
## labeventsiddd = pd.read_csv('labevents.csv',usecols=use)
## 158374764 rows × 1 columns

In [72]:
import pandas as pd

# Path to labevents file
labevents_path = 'labevents.csv'

# Relevant itemids for sepsis features
"""lab_features_itemid"""

required_labf = ['subject_id', 'hadm_id', 'charttime', 'itemid', 'valuenum', 'valueuom']

# Initialize a list to store filtered data
labfeatures_s = []

# Read labevents in chunks
for chunk in pd.read_csv(labevents_path, usecols=required_labf , chunksize=5000000):
    # Filter rows based on relevant itemids
    filtered_chunk = chunk[chunk['itemid'].isin(lab_features_itemid)]
    # Append filtered data
    labfeatures_s.append(filtered_chunk)

# Combine all chunks into a single DataFrame
labfeatures_final = pd.concat(labfeatures_s, ignore_index=True)

# Save the filtered data to a new CSV file
labfeatures_final.to_csv('filtered_labevents.csv', index=False)

print("Filtered labevents saved to 'filtered_labevents.csv'")

Filtered labevents saved to 'filtered_labevents.csv'


In [77]:
filtered_labevents = pd.read_csv("filtered_labevents.csv")
filtered_labevents.head(5)    # 17474675 rows × 6 columns

,subject_id,hadm_id,itemid,charttime,valuenum,valueuom
0,10000032,NaN,50882,2180-03-23 11:51:00,27.0,mEq/L
1,10000032,NaN,50912,2180-03-23 11:51:00,0.4,mg/dL
2,10000032,NaN,51265,2180-03-23 11:51:00,83.0,K/uL
3,10000032,NaN,51301,2180-03-23 11:51:00,3.0,K/uL
4,10000032,NaN,51265,2180-05-06 22:25:00,71.0,K/uL


In [79]:
# Merge filtered labevents with d_labitems on 'itemid'
merged_labevents = filtered_labevents.merge(labitems, on='itemid', how='left')
merged_labevents.head(5)

,subject_id,hadm_id,itemid,charttime,valuenum,valueuom,label,fluid,category
0,10000032,NaN,50882,2180-03-23 11:51:00,27.0,mEq/L,Bicarbonate,Blood,Chemistry
1,10000032,NaN,50912,2180-03-23 11:51:00,0.4,mg/dL,Creatinine,Blood,Chemistry
2,10000032,NaN,51265,2180-03-23 11:51:00,83.0,K/uL,Platelet Count,Blood,Hematology
3,10000032,NaN,51301,2180-03-23 11:51:00,3.0,K/uL,White Blood Cells,Blood,Hematology
4,10000032,NaN,51265,2180-05-06 22:25:00,71.0,K/uL,Platelet Count,Blood,Hematology


In [97]:
final_labevents = merged_labevents.pivot_table(index=['subject_id', 'hadm_id'], columns='label',values='valuenum',aggfunc='mean').reset_index()
final_labevents

label,subject_id,hadm_id,Bicarbonate,C-Reactive Protein,Creatinine,Lactate,Platelet Count,White Blood Cells
0,10000032,22595853.0,28.000000,NaN,0.300000,NaN,71.000000,4.200000
1,10000032,22841357.0,25.000000,NaN,0.300000,NaN,137.000000,6.600000
2,10000032,25742920.0,25.000000,NaN,0.466667,NaN,120.000000,6.550000
3,10000032,29079034.0,24.000000,NaN,0.433333,NaN,94.500000,4.450000
4,10000084,23052089.0,23.000000,NaN,0.720000,NaN,284.200000,8.000000
...,...,...,...,...,...,...,...,...
432795,19999828,25744818.0,22.200000,NaN,0.640000,1.50,391.300000,10.110000
432796,19999828,29734428.0,24.842105,NaN,0.568421,1.90,369.736842,10.000000
432797,19999840,21033226.0,26.250000,NaN,0.708333,3.66,277.333333,14.922222
432798,19999840,26071774.0,25.250000,NaN,0.800000,NaN,208.000000,15.725000


In [98]:
final_labevents.isnull().sum()

label
subject_id                 0
hadm_id                    0
Bicarbonate            25905
C-Reactive Protein    399898
Creatinine             16967
Lactate               321266
Platelet Count          8814
White Blood Cells      11250
dtype: int64

In [102]:
missing_value_cols = ['Bicarbonate', 'C-Reactive Protein', 'Creatinine', 'Lactate', 'Platelet Count', 'White Blood Cells']

for column in missing_value_cols:
    median_value = final_labevents[column].median()
    final_labevents[column] = final_labevents[column].fillna(median_value)
final_labevents.isnull().sum()

label
subject_id            0
hadm_id               0
Bicarbonate           0
C-Reactive Protein    0
Creatinine            0
Lactate               0
Platelet Count        0
White Blood Cells     0
dtype: int64

In [103]:
final_labevents

label,subject_id,hadm_id,Bicarbonate,C-Reactive Protein,Creatinine,Lactate,Platelet Count,White Blood Cells
0,10000032,22595853.0,28.000000,39.6,0.300000,1.60,71.000000,4.200000
1,10000032,22841357.0,25.000000,39.6,0.300000,1.60,137.000000,6.600000
2,10000032,25742920.0,25.000000,39.6,0.466667,1.60,120.000000,6.550000
3,10000032,29079034.0,24.000000,39.6,0.433333,1.60,94.500000,4.450000
4,10000084,23052089.0,23.000000,39.6,0.720000,1.60,284.200000,8.000000
...,...,...,...,...,...,...,...,...
432795,19999828,25744818.0,22.200000,39.6,0.640000,1.50,391.300000,10.110000
432796,19999828,29734428.0,24.842105,39.6,0.568421,1.90,369.736842,10.000000
432797,19999840,21033226.0,26.250000,39.6,0.708333,3.66,277.333333,14.922222
432798,19999840,26071774.0,25.250000,39.6,0.800000,1.60,208.000000,15.725000


In [105]:
# final_labevents.to_csv("sepsis_labevents.csv", index=False)

### 2. chartevents.csv and d_chartitems.csv

In [56]:
## chartevents = pd.read_csv('chartevents.csv',usecols=['warning'])
## 432997491 rows × 1 columns

**Chart items ids extraction**

In [31]:
chartitems = pd.read_csv("d_items.csv")

In [32]:
chartitems.head(2)

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN


In [6]:
chartitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4095 entries, 0 to 4094
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   itemid           4095 non-null   int64  
 1   label            4095 non-null   object 
 2   abbreviation     4095 non-null   object 
 3   linksto          4095 non-null   object 
 4   category         4095 non-null   object 
 5   unitname         1123 non-null   object 
 6   param_type       4095 non-null   object 
 7   lownormalvalue   19 non-null     float64
 8   highnormalvalue  22 non-null     float64
dtypes: float64(2), int64(1), object(6)
memory usage: 288.1+ KB


In [20]:
keywords = ['heart rate', 'blood pressure', 'temperature', 'respiratory', 'oxygen', 'gcs', 'glucose']

req_itemids = chartitems[chartitems['label'].str.contains('|'.join(keywords), case=False, na=False)]
required_itemids = req_itemids[['itemid']]

In [27]:
required_itemids.head()

,itemid
2,220045
3,220046
4,220047
6,220050
7,220051


In [43]:
req_itemids['label'].unique()

array(['Heart Rate', 'Heart rate Alarm - High', 'Heart Rate Alarm - Low',
       'Arterial Blood Pressure systolic',
       'Arterial Blood Pressure diastolic',
       'Arterial Blood Pressure mean',
       'Arterial Blood Pressure Alarm - Low',
       'Arterial Blood Pressure Alarm - High',
       'Non Invasive Blood Pressure systolic',
       'Non Invasive Blood Pressure diastolic',
       'Non Invasive Blood Pressure mean', 'Respiratory Rate',
       'Glucose (ingr)', 'Glucose (serum)', 'GCS - Eye Opening',
       'Non-Invasive Blood Pressure Alarm - High',
       'Non-Invasive Blood Pressure Alarm - Low',
       'Temperature Fahrenheit', 'Temperature Celsius',
       'GCS - Verbal Response', 'GCS - Motor Response',
       'Respiratory Pattern', 'Respiratory Effort', 'Skin Temperature',
       'Manual Blood Pressure Systolic Left', 'Temperature Site',
       'Manual Blood Pressure Diastolic Left', 'Changes in Temperature',
       'Respiratory Rate (Set)', 'Respiratory Rate (spontane

**Extracting Chartevent Features**

In [28]:
import pandas as pd

chart_itemids = req_itemids['itemid'].unique()
required_columns = ['subject_id', 'hadm_id', 'itemid', 'valuenum', 'valueuom']

# Process chart events in chunks
chart_events_chunks = pd.read_csv('chartevents.csv', usecols=required_columns, chunksize=5000000)

filtered_rows = []
for chunk in chart_events_chunks:
    filtered_chunk = chunk[chunk['itemid'].isin(chart_itemids)]
    filtered_rows.append(filtered_chunk)

# Combine all filtered chunks
filtered_chart_events = pd.concat(filtered_rows, ignore_index=True)
filtered_chart_events.head(5)

,subject_id,hadm_id,itemid,valuenum,valueuom
0,10000032,29079034,224642,NaN,NaN
1,10000032,29079034,223761,98.7,°F
2,10000032,29079034,220179,84.0,mmHg
3,10000032,29079034,220180,48.0,mmHg
4,10000032,29079034,220181,56.0,mmHg


In [30]:
# Save the filtered data
filtered_chart_events.to_csv('filtered_chartevents.csv', index=False)

In [29]:
filtered_chart_events

,subject_id,hadm_id,itemid,valuenum,valueuom
0,10000032,29079034,224642,NaN,NaN
1,10000032,29079034,223761,98.7,°F
2,10000032,29079034,220179,84.0,mmHg
3,10000032,29079034,220180,48.0,mmHg
4,10000032,29079034,220181,56.0,mmHg
...,...,...,...,...,...
69264291,19999987,23865745,220179,110.0,mmHg
69264292,19999987,23865745,220180,70.0,mmHg
69264293,19999987,23865745,225664,127.0,NaN
69264294,19999987,23865745,220621,113.0,mg/dL


In [35]:
merged_chartevents = filtered_chart_events.merge(chartitems[['itemid', 'label']], on='itemid', how='left')
merged_chartevents.head(2)

,subject_id,hadm_id,itemid,valuenum,valueuom,label
0,10000032,29079034,224642,NaN,NaN,Temperature Site
1,10000032,29079034,223761,98.7,°F,Temperature Fahrenheit


In [36]:
merged_chartevents

,subject_id,hadm_id,itemid,valuenum,valueuom,label
0,10000032,29079034,224642,NaN,NaN,Temperature Site
1,10000032,29079034,223761,98.7,°F,Temperature Fahrenheit
2,10000032,29079034,220179,84.0,mmHg,Non Invasive Blood Pressure systolic
3,10000032,29079034,220180,48.0,mmHg,Non Invasive Blood Pressure diastolic
4,10000032,29079034,220181,56.0,mmHg,Non Invasive Blood Pressure mean
...,...,...,...,...,...,...
69264291,19999987,23865745,220179,110.0,mmHg,Non Invasive Blood Pressure systolic
69264292,19999987,23865745,220180,70.0,mmHg,Non Invasive Blood Pressure diastolic
69264293,19999987,23865745,225664,127.0,NaN,Glucose finger stick (range 70-100)
69264294,19999987,23865745,220621,113.0,mg/dL,Glucose (serum)


In [37]:
merged_chartevents['label'].value_counts()

label
Heart Rate                               8752069
Respiratory Rate                         8636655
Non Invasive Blood Pressure systolic     5378740
Non Invasive Blood Pressure diastolic    5377689
Non Invasive Blood Pressure mean         5372922
                                          ...   
GcsScore_ApacheIV                              8
Glucose_ApacheIV                               8
TemperatureF_ApacheIV                          7
Plan-Respiratory                               7
Respiratory Quotient                           2
Name: count, Length: 66, dtype: int64

In [53]:
# Define grouping keywords
group_keywords = {
    'Heart Rate': [
        'Heart Rate', 'Heart rate Alarm - High', 'Heart Rate Alarm - Low'],
    'Blood Pressure': [
        'Arterial Blood Pressure systolic', 'Arterial Blood Pressure diastolic',
        'Arterial Blood Pressure mean', 'Arterial Blood Pressure Alarm - Low',
        'Arterial Blood Pressure Alarm - High', 'Non Invasive Blood Pressure systolic',
        'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean',
        'Manual Blood Pressure Systolic Left', 'Manual Blood Pressure Diastolic Left',
        'Manual Blood Pressure Diastolic Right', 'Manual Blood Pressure Systolic Right',
        'ART Blood Pressure Alarm - High', 'ART Blood Pressure Alarm - Low',
        'ART Blood Pressure Alarm Source'],
    'Respiratory': [
        'Respiratory Rate', 'Respiratory Rate (Set)', 'Respiratory Rate (spontaneous)',
        'Respiratory Rate (Total)', 'Respiratory Pattern', 'Respiratory Effort',
        'Respiratory Arrest', 'Respiratory Quotient', 'Pre-Oxygentated (Intubation)',
        'Respiratory', 'Altered Respiratory Status NCP - Expected outcomes',
        'Altered Respiratory Status NCP - Goal',
        'Altered Respiratory Status NCP - Interventions',
        'Altered Respiratory Status NCP - outcomes met',
        'Altered Respiratory Status NCP - Plan revised',
        'Altered Respiratory Status NCP - Problem resolved',
        'ROS-Respiratory', 'Plan-Respiratory', 'Post-Operative Respiratory (RESPIRAT)',
        'Non-Operative Respiratory (RESPIRAT)'],
    'Oxygen': [
        'OxygenApacheIIScore', 'OxygenScore_ApacheIV', 'PAR-Oxygen saturation',
        'Oxygenator Sweep Rate', 'Oxygenator/ECMO', 'Oxygenator visible (ECMO)',
        'Oxygenator visible (CH)', 'Preoxygenation (Intubation)'],
    'Glucose': [
        'Glucose (ingr)', 'Glucose (serum)', 'Glucose (whole blood)',
        'Glucose finger stick (range 70-100)', 'Glucose_ApacheIV', 'GlucoseScore_ApacheIV',
        'Glucose Control - Prophy', 'Boost Glucose Control (1/4)',
        'Boost Glucose Control (1/2)', 'Boost Glucose Control (3/4)',
        'Boost Glucose Control (Full)'],
    'Temperature': [
        'Temperature Fahrenheit', 'Temperature Celsius', 'TemperatureF_ApacheIV',
        'Skin Temperature', 'Changes in Temperature', 'Cerebral Temperature (C)',
        'Pt. Temperature (BG) (SOFT)', 'Blood Temperature CCO (C)', 'Temperature Site'],
    'GCS': [
        'GCS - Eye Opening', 'GCS - Verbal Response', 'GCS - Motor Response',
        'GcsApacheIIScore', 'GCSEyeApacheIIValue', 'GCSMotorApacheIIValue',
        'GCSVerbalApacheIIValue', 'GCSEye_ApacheIV', 'GCSMotor_ApacheIV',
        'GcsScore_ApacheIV', 'GCSVerbal_ApacheIV', 'GCSVerbalApacheIIValue (intubated)'],
    'Brain': [
        'Brain Tissue Oxygenation']}


# Create a mapping dictionary for labels
label_mapping = {}
for group, keywords in group_keywords.items():
    for keyword in keywords:
        label_mapping[keyword.lower()] = group
# Map the labels to their respective groups
merged_chartevents['label_grouped'] = merged_chartevents['label'].str.lower().map(
    lambda x: next((group for keyword, group in label_mapping.items() if keyword in x), 'Other')
)

In [55]:
merged_chartevents['label_grouped'].value_counts()

label_grouped
Blood Pressure    26020237
Respiratory       16661146
Heart Rate        10439144
GCS                6614331
Temperature        6387211
Glucose            1817307
Other              1268863
Oxygen               51549
Brain                 4508
Name: count, dtype: int64

In [57]:
# Drop the original 'label' column as it's no longer needed
merged_chartevents = merged_chartevents.drop('label', axis=1)

# Pivot the data directly to aggregate using mean
pivot_chartevents = merged_chartevents.pivot_table(index=['subject_id', 'hadm_id'], columns='label_grouped', values='valuenum', aggfunc='mean' ).reset_index()
pivot_chartevents

label_grouped,subject_id,hadm_id,Blood Pressure,Brain,GCS,Glucose,Heart Rate,Other,Oxygen,Respiratory,Temperature
0,10000032,29079034,68.433333,NaN,4.888889,115.000000,94.642857,121.250000,NaN,20.700000,98.966667
1,10000690,25860671,86.063830,NaN,4.694444,100.333333,84.190909,125.000000,NaN,22.557895,97.736364
2,10000980,26913865,107.757576,NaN,5.000000,109.500000,73.636364,125.000000,NaN,21.538462,98.133333
3,10001217,24597018,103.837500,NaN,5.000000,113.000000,92.696970,120.000000,NaN,21.259259,99.066667
4,10001217,27703517,90.855072,NaN,5.000000,129.000000,81.612903,120.000000,NaN,16.640000,98.133333
...,...,...,...,...,...,...,...,...,...,...,...
85237,19999442,26785317,104.250000,NaN,3.742857,120.321429,63.037915,130.666667,1.777778,15.811224,98.548837
85238,19999625,25304202,80.863636,NaN,4.200000,147.000000,72.076923,120.000000,NaN,18.480000,98.466667
85239,19999828,25744818,89.928571,NaN,4.833333,215.000000,94.076923,125.000000,NaN,14.380952,98.320000
85240,19999840,21033226,73.957230,NaN,1.411765,149.178571,75.700000,125.312500,NaN,14.190647,97.706250


In [58]:
pivot_chartevents.isnull().sum()

label_grouped
subject_id            0
hadm_id               0
Blood Pressure       55
Brain             85195
GCS                 201
Glucose            1562
Heart Rate            1
Other              3964
Oxygen            79043
Respiratory          74
Temperature         338
dtype: int64

In [60]:
# Drop the 'Other' column if it exists
pivot_chartevents = pivot_chartevents.drop(columns=['Other'], errors='ignore',axis=1)

# Define the columns with missing values (excluding 'Other')
missing_value_colus = ['Blood Pressure', 'Brain', 'GCS', 'Glucose', 
                      'Heart Rate', 'Oxygen', 'Respiratory', 'Temperature']

# Fill missing values with the median of each column
for column in missing_value_colus:
    median_value = pivot_chartevents[column].median()
    pivot_chartevents[column] = pivot_chartevents[column].fillna(median_value)

# Check for remaining missing values
pivot_chartevents.isnull().sum()

label_grouped
subject_id        0
hadm_id           0
Blood Pressure    0
Brain             0
GCS               0
Glucose           0
Heart Rate        0
Oxygen            0
Respiratory       0
Temperature       0
dtype: int64

In [61]:
pivot_chartevents.to_csv('sepsis_chartevents.csv', index=False)

### Merging Filtered Labevents and Chartevents

In [62]:
labevents = pd.read_csv("sepsis_labevents.csv")
chartevents = pd.read_csv("sepsis_chartevents.csv")
labchart_events = pd.merge(labevents, chartevents, on=['subject_id', 'hadm_id'], how='inner')

labchart_events.head()

,subject_id,hadm_id,Bicarbonate,C-Reactive Protein,Creatinine,Lactate,Platelet Count,White Blood Cells,Blood Pressure,Brain,GCS,Glucose,Heart Rate,Oxygen,Respiratory,Temperature
0,10000032,29079034.0,24.000000,39.6,0.433333,1.6,94.500000,4.450000,68.433333,25.07069,4.888889,115.000000,94.642857,1.0,20.700000,98.966667
1,10000690,25860671.0,32.272727,39.6,0.781818,1.6,242.200000,5.730000,86.063830,25.07069,4.694444,100.333333,84.190909,1.0,22.557895,97.736364
2,10000980,26913865.0,23.714286,39.6,2.071429,1.6,204.333333,5.166667,107.757576,25.07069,5.000000,109.500000,73.636364,1.0,21.538462,98.133333
3,10001217,24597018.0,26.000000,39.6,0.500000,1.6,349.333333,13.150000,103.837500,25.07069,5.000000,113.000000,92.696970,1.0,21.259259,99.066667
4,10001217,27703517.0,27.800000,39.6,0.460000,1.6,319.800000,7.640000,90.855072,25.07069,5.000000,129.000000,81.612903,1.0,16.640000,98.133333


In [65]:
labchart_events.isnull().sum()

subject_id            0
hadm_id               0
Bicarbonate           0
C-Reactive Protein    0
Creatinine            0
Lactate               0
Platelet Count        0
White Blood Cells     0
Blood Pressure        0
Brain                 0
GCS                   0
Glucose               0
Heart Rate            0
Oxygen                0
Respiratory           0
Temperature           0
dtype: int64

In [69]:
list(labchart_events.columns)

['subject_id',
 'hadm_id',
 'Bicarbonate',
 'C-Reactive Protein',
 'Creatinine',
 'Lactate',
 'Platelet Count',
 'White Blood Cells',
 'Blood Pressure',
 'Brain',
 'GCS',
 'Glucose',
 'Heart Rate',
 'Oxygen',
 'Respiratory',
 'Temperature']

In [119]:
labchart_events.shape

(84325, 16)

In [120]:
labchart_events.to_csv('sepsis_labchartevents.csv', index=False)
print("sepsis_labchart_events saved")

sepsis_labchart_events saved


### 3. admissions.csv and patients.csv

In [72]:
admissions = pd.read_csv("admissions.csv")
print(admissions.shape)
admissions.head(2)

(546028, 16)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0


In [74]:
patients = pd.read_csv("patients.csv")
print(patients.shape)
patients.head(2)

(364627, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN


#### Process admissions table

In [108]:
# Selecting relevant columns
admissions_filtered = admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime',
                                  'admission_type', 'admission_location', 
                                  'discharge_location', 'insurance', 'hospital_expire_flag']].copy()

# Calculate hospital stay duration in hours
admissions_filtered['hospital_stay_hours'] = (
    pd.to_datetime(admissions_filtered['dischtime']) - pd.to_datetime(admissions_filtered['admittime'])
).dt.total_seconds() / 3600

# Drop rows with negative or null stay durations (invalid data)
admissions_filtered = admissions_filtered[admissions_filtered['hospital_stay_hours'] > 0]
print(admissions_filtered.shape)
admissions_filtered.head(2)

(545848, 10)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,hospital_expire_flag,hospital_stay_hours
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,0,18.866667
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,0,24.366667


#### Process patients table

In [112]:
# Select relevant columns
patients_filtered = patients[['subject_id', 'gender', 'anchor_age']].copy()

# Handle missing values if any
# Fill NaN values and reassign explicitly
patients_filtered['anchor_age'] = patients_filtered['anchor_age'].fillna(patients_filtered['anchor_age'].median())
patients_filtered['gender'] = patients_filtered['gender'].fillna('Unknown')
print(patients_filtered.shape)
patients_filtered.head(2)

(364627, 3)


,subject_id,gender,anchor_age
0,10000032,F,52
1,10000048,F,23


### 4. diagnoses_icd.csv and d_icd_diagnoses.csv

In [76]:
d_icd_diagnosis = pd.read_csv("d_icd_diagnoses.csv")
print(d_icd_diagnosis.shape)
d_icd_diagnosis.head(2)

(112107, 3)


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor


In [77]:
diagnosis_icd = pd.read_csv("diagnoses_icd.csv")
print(diagnosis_icd.shape)
diagnosis_icd.head(2)

(6364488, 5)


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9


#### Sepsis - Target Extraction

In [82]:
diagnosis_combined = pd.merge(diagnosis_icd, d_icd_diagnosis, on='icd_code', how='left')
diagnosis_combined

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
0,10000032,22595853,1,5723,9,9,Portal hypertension
1,10000032,22595853,2,78959,9,9,Other ascites
2,10000032,22595853,3,5715,9,9,Cirrhosis of liver without mention of alcohol
3,10000032,22595853,4,07070,9,9,Unspecified viral hepatitis C without hepatic ...
4,10000032,22595853,5,496,9,9,"Chronic airway obstruction, not elsewhere clas..."
...,...,...,...,...,...,...,...
6484223,19999987,23865745,7,41401,9,9,Coronary atherosclerosis of native coronary ar...
6484224,19999987,23865745,8,78039,9,9,Other convulsions
6484225,19999987,23865745,9,0413,9,9,Friedländer's bacillus infection in conditions...
6484226,19999987,23865745,10,36846,9,9,Homonymous bilateral field defects


In [87]:
diagnosis_combined['long_title'].value_counts()

long_title
Unspecified essential hypertension                                                                             102368
Hyperlipidemia, unspecified                                                                                     84570
Essential (primary) hypertension                                                                                83775
Other and unspecified hyperlipidemia                                                                            67293
Acute kidney failure, unspecified                                                                               65020
                                                                                                                ...  
Laceration of extensor muscle, fascia and tendon of right little finger at forearm level, initial encounter         1
Stress fracture, right tibia, subsequent encounter for fracture with routine healing                                1
Congenital pes cavus                         

In [101]:
diagnosis_combined = diagnosis_combined.drop(['seq_num','icd_version_x',"icd_version_y"],axis=1)
sepsis_icd_keywords = diagnosis_combined[diagnosis_combined['long_title'].str.contains('sepsis|septicemia|SIRS|septic shock', case=False, na=False)]

In [103]:
print(sepsis_icd_keywords.shape)
sepsis_icd_keywords.head(2)

(42844, 4)


,subject_id,hadm_id,icd_code,long_title
206,10000826,21086876,99591,Sepsis
626,10001401,27012892,A4181,Sepsis due to Enterococcus


In [105]:
# Create a column for sepsis diagnosis
sepsis_icd_keywords.loc[:, 'sepsis'] = 1
# Keep only relevant columns
sepsis_presence = sepsis_icd_keywords[['subject_id', 'hadm_id', 'sepsis']]
print(sepsis_presence.shape)
sepsis_presence.head(2)

(42844, 3)


,subject_id,hadm_id,sepsis
206,10000826,21086876,1
626,10001401,27012892,1


In [106]:
sepsis_presence.to_csv('sepsis_presence.csv', index=False)

### 5. icustays.csv

In [78]:
icu_stays = pd.read_csv("icustays.csv")
print(icu_stays.shape)
icu_stays.head(2)

(94458, 8)


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252


In [113]:
# Select relevant columns
icustays_filtered = icu_stays[['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 
                              'last_careunit', 'intime', 'outtime', 'los']].copy()

# Calculate ICU stay duration in hours
icustays_filtered['icu_stay_hours'] = (
    pd.to_datetime(icustays_filtered['outtime']) - pd.to_datetime(icustays_filtered['intime'])
).dt.total_seconds() / 3600

# Drop rows with invalid ICU durations
icustays_filtered = icustays_filtered[icustays_filtered['icu_stay_hours'] > 0]
print(icustays_filtered.shape)
icustays_filtered.head(2)

(94444, 9)


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,icu_stay_hours
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,9.846389
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056


### Processed Tables + Target

In [115]:
# Step 1: Merge admissions data
intermediate_data = pd.merge(admissions_filtered, patients_filtered, on='subject_id', how='left')

# Step 2: Merge ICU stays data
intermediate_data = pd.merge(intermediate_data, icustays_filtered, on=['subject_id', 'hadm_id'], how='left')

# Step 3: Merge sepsis diagnosis data
intermediate_data = pd.merge(intermediate_data, sepsis_presence, on=['subject_id', 'hadm_id'], how='left')

# Fill NaN values for sepsis_flag
# Explicitly reassign after filling NaN
intermediate_data['sepsis'] = intermediate_data['sepsis'].fillna(0)

In [116]:
print(intermediate_data.shape)
intermediate_data.head(2)

(578317, 20)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,hospital_expire_flag,hospital_stay_hours,gender,anchor_age,stay_id,first_careunit,last_careunit,intime,outtime,los,icu_stay_hours,sepsis
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,0,18.866667,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,0,24.366667,F,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [117]:
intermediate_data['sepsis'].value_counts()

sepsis
0.0    529165
1.0     49152
Name: count, dtype: int64

In [118]:
# Save the intermediate dataset
intermediate_data.to_csv('sepsis_patient_data.csv', index=False)
print('sepsis data saved')

sepsis data saved


# Final Dataset - labevents + chartevents + patients data + Target

In [3]:
labchart_events = pd.read_csv("sepsis_labchartevents.csv")
patients_data = pd.read_csv("sepsis_patient_data.csv")

In [4]:
patients_data['icu_stay_hours'].isnull().sum()

464992

In [5]:
print("Labevents+Chartevents")
print(labchart_events.columns)
print("Patients Data & Target")
patients_data.columns

Labevents+Chartevents
Index(['subject_id', 'hadm_id', 'Bicarbonate', 'C-Reactive Protein',
       'Creatinine', 'Lactate', 'Platelet Count', 'White Blood Cells',
       'Blood Pressure', 'Brain', 'GCS', 'Glucose', 'Heart Rate', 'Oxygen',
       'Respiratory', 'Temperature'],
      dtype='object')
Patients Data & Target


Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance',
       'hospital_expire_flag', 'hospital_stay_hours', 'gender', 'anchor_age',
       'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime',
       'los', 'icu_stay_hours', 'sepsis'],
      dtype='object')

In [6]:
# Merge lab-chart data with patient data
sepsis_prediction_dataset = pd.merge(patients_data,labchart_events,on=['subject_id', 'hadm_id'],how='inner')
print(sepsis_prediction_dataset.shape)
sepsis_prediction_dataset.head(2)

(112349, 34)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,hospital_expire_flag,hospital_stay_hours,...,Platelet Count,White Blood Cells,Blood Pressure,Brain,GCS,Glucose,Heart Rate,Oxygen,Respiratory,Temperature
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,0,53.333333,...,94.5,4.45,68.433333,25.07069,4.888889,115.000000,94.642857,1.0,20.700000,98.966667
1,10000690,25860671,2150-11-02 18:02:00,2150-11-12 13:45:00,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,0,235.716667,...,242.2,5.73,86.063830,25.07069,4.694444,100.333333,84.190909,1.0,22.557895,97.736364


In [7]:
sepsis_prediction_dataset.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance',
       'hospital_expire_flag', 'hospital_stay_hours', 'gender', 'anchor_age',
       'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime',
       'los', 'icu_stay_hours', 'sepsis', 'Bicarbonate', 'C-Reactive Protein',
       'Creatinine', 'Lactate', 'Platelet Count', 'White Blood Cells',
       'Blood Pressure', 'Brain', 'GCS', 'Glucose', 'Heart Rate', 'Oxygen',
       'Respiratory', 'Temperature'],
      dtype='object')

In [8]:
sepsis_prediction_dataset.isnull().sum()

subject_id                 0
hadm_id                    0
admittime                  0
dischtime                  0
admission_type             0
admission_location         0
discharge_location       750
insurance               1626
hospital_expire_flag       0
hospital_stay_hours        0
gender                     0
anchor_age                 0
stay_id                    8
first_careunit             8
last_careunit              8
intime                     8
outtime                    8
los                        8
icu_stay_hours             8
sepsis                     0
Bicarbonate                0
C-Reactive Protein         0
Creatinine                 0
Lactate                    0
Platelet Count             0
White Blood Cells          0
Blood Pressure             0
Brain                      0
GCS                        0
Glucose                    0
Heart Rate                 0
Oxygen                     0
Respiratory                0
Temperature                0
dtype: int64

In [11]:
sepsis_prediction_dataset['sepsis'].value_counts()

sepsis
0.0    76192
1.0    36157
Name: count, dtype: int64

In [13]:
# Fill missing numerical values with median
numerical_columns = sepsis_prediction_dataset.select_dtypes(include=['number']).columns
for col in numerical_columns:
    sepsis_prediction_dataset[col] = sepsis_prediction_dataset[col].fillna(sepsis_prediction_dataset[col].median())


# Fill missing categorical values with mode
categorical_columns = sepsis_prediction_dataset.select_dtypes(include=['object', 'category']).columns
for col in categorical_columns:
    sepsis_prediction_dataset[col] = sepsis_prediction_dataset[col].fillna(sepsis_prediction_dataset[col].mode()[0])

In [14]:
sepsis_prediction_dataset.isnull().sum()

subject_id              0
hadm_id                 0
admittime               0
dischtime               0
admission_type          0
admission_location      0
discharge_location      0
insurance               0
hospital_expire_flag    0
hospital_stay_hours     0
gender                  0
anchor_age              0
stay_id                 0
first_careunit          0
last_careunit           0
intime                  0
outtime                 0
los                     0
icu_stay_hours          0
sepsis                  0
Bicarbonate             0
C-Reactive Protein      0
Creatinine              0
Lactate                 0
Platelet Count          0
White Blood Cells       0
Blood Pressure          0
Brain                   0
GCS                     0
Glucose                 0
Heart Rate              0
Oxygen                  0
Respiratory             0
Temperature             0
dtype: int64

In [16]:
print(sepsis_prediction_dataset.shape)
sepsis_prediction_dataset.head(2)

(112349, 34)


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,hospital_expire_flag,hospital_stay_hours,...,Platelet Count,White Blood Cells,Blood Pressure,Brain,GCS,Glucose,Heart Rate,Oxygen,Respiratory,Temperature
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,0,53.333333,...,94.5,4.45,68.433333,25.07069,4.888889,115.000000,94.642857,1.0,20.700000,98.966667
1,10000690,25860671,2150-11-02 18:02:00,2150-11-12 13:45:00,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,0,235.716667,...,242.2,5.73,86.063830,25.07069,4.694444,100.333333,84.190909,1.0,22.557895,97.736364


In [17]:
sepsis_prediction_dataset.to_csv('sepsis_prediction_dataset.csv', index=False)
print("sepsis prediction dataset saved")

sepsis prediction dataset saved


In [18]:
sepsis_prediction_dataset.dtypes

subject_id                int64
hadm_id                   int64
admittime                object
dischtime                object
admission_type           object
admission_location       object
discharge_location       object
insurance                object
hospital_expire_flag      int64
hospital_stay_hours     float64
gender                   object
anchor_age                int64
stay_id                 float64
first_careunit           object
last_careunit            object
intime                   object
outtime                  object
los                     float64
icu_stay_hours          float64
sepsis                  float64
Bicarbonate             float64
C-Reactive Protein      float64
Creatinine              float64
Lactate                 float64
Platelet Count          float64
White Blood Cells       float64
Blood Pressure          float64
Brain                   float64
GCS                     float64
Glucose                 float64
Heart Rate              float64
Oxygen  

In [28]:
sepsis_prediction_dataset['admission_location'].value_counts()

admission_location
EMERGENCY ROOM                            46074
TRANSFER FROM HOSPITAL                    29544
PHYSICIAN REFERRAL                        25903
WALK-IN/SELF REFERRAL                      5394
TRANSFER FROM SKILLED NURSING FACILITY     1985
CLINIC REFERRAL                            1472
PROCEDURE SITE                             1150
PACU                                        456
INFORMATION NOT AVAILABLE                   255
AMBULATORY SURGERY TRANSFER                  88
INTERNAL TRANSFER TO OR FROM PSYCH           28
Name: count, dtype: int64